In [52]:
import pandas as pd
import numpy as np

In [2]:
def to_csv(df, path):
    # Prepend dtypes to the top of df
    df2 = df.copy()
    df2.loc[-1] = df2.dtypes
    df2.index = df2.index + 1
    df2.sort_index(inplace=True)
    # Then save it to a csv
    df2.to_csv(path, index=False)

def read_csv(path):
    # Read types first line of csv
    dtypes = {key:value for (key,value) in pd.read_csv(path,    
              nrows=1).iloc[0].to_dict().items() if 'date' not in value}

    parse_dates = [key for (key,value) in pd.read_csv(path, 
                   nrows=1).iloc[0].to_dict().items() if 'date' in value]
    # Read the rest of the lines with the types from above
    return pd.read_csv(path, dtype=dtypes, parse_dates=parse_dates, skiprows=[1])

In [3]:
dtypes = {'date': 'string',
 'sity': 'string',
 'market': 'string',
 'discounts': 'UInt32',
 'data_uuid': 'string',
 'href': 'string',
 'su1': 'string',
 'su2': 'string',
 'su3': 'string',
 'title': 'string',
 'sku_id': 'string',
 'dateStart': 'string',
 'dateEnd': 'string',
 'price': 'UInt32',
 'price_from': 'UInt32',
 'price_to': 'UInt32',
 'discountPercent': 'UInt32',
 'quantity': 'Float32',
 'quantityUnit': 'string'}

In [5]:
df = pd.read_csv('data/result_lipetsk_df.zip', dtype=dtypes)
for col in ['date', 'dateStart', 'dateEnd']:
    df[col] = pd.to_datetime(df[col])

In [6]:
df = df[df['su1'] == 'Продукты'].reset_index(drop=True)

In [82]:
def fraction(df):
    def worker(x):
        if pd.isnull(x['quantity']):
            return np.nan
        if not pd.isnull(x['price']):
            return int((x['price']*10)/x['quantity'])
        else:
            return int((x['price_to']*10)/x['quantity'])
    return df.agg(worker, axis=1).astype('UInt16')

In [138]:
mask_su2 = df['su2'] == 'Молочные продукты'
mask_su3 = df['su3'] == 'Молоко'
mask1 = df['title'].str.lower().str.match('.*(молоко).*(лебед).*').fillna(False)
mask2 = df['quantity'] > 1000
mask = (
    mask_su2
    & mask_su3
    & mask1
    & mask2
    )
with pd.option_context("display.max_columns", 500, "display.max_colwidth", 1000):
    display(
                df[mask][['market', 'title', 'price', 'price_from', 'price_to', 'quantity', 'discountPercent']]
                .assign(fract=fraction)
                .sort_values(by='fract', ascending=True)
                .head(3)
            )

,market,title,price,price_from,price_to,quantity,discountPercent,fract
7194,Пятёрочка,Молоко Лебедяньмолоко пастеризованное 2.5% 1400г,10000,10300,<NA>,1400.0,0,71
10773,Правильная корзинка,"Бзмж Молоко ""лебедяньмолоко"" 3,2% Пэт 1400 Г",<NA>,<NA>,11448,1400.0,10,81


In [136]:
mask_su2 = df['su2'] == 'Молочные продукты'
mask_su3 = df['su3'] == 'Творог'
mask1 = df['title'].str.lower().str.match('.*(творог).*(лебед).*').fillna(False)
mask2 = df['quantity'] > 250
mask = (
    mask_su2
    & mask_su3
    & mask1
    & mask2
    )
with pd.option_context("display.max_columns", 500, "display.max_colwidth", 1000):
    display(
                df[mask][['market', 'title', 'price', 'price_from', 'price_to', 'quantity', 'discountPercent']]
                .assign(fract=fraction)
                .sort_values(by='fract', ascending=True)
                .head(3)
            )


,market,title,price,price_from,price_to,quantity,discountPercent,fract
10578,Metro,"Творог Лебедяньмолоко 5%, 300г Россия",<NA>,<NA>,9990,300.0,13,333
9032,Европа,Творог 300г 5% Лебедяньмолоко,<NA>,<NA>,10999,300.0,27,366
29005,Липка,Творог 5% 300г Лебедянь,<NA>,<NA>,10990,300.0,22,366


In [134]:

# mask_su2 = df['su2'] == 'Молочные продукты'
# mask_su3 = df['su3'] == 'Творог'
mask1 = df['title'].str.lower().str.match('.*(m\&m).*').fillna(False)
# mask2 = df['quantity'] > 250
mask = (
    # mask_su2
    # & mask_su3
    mask1
    # & mask2
    )
with pd.option_context("display.max_columns", 500, "display.max_colwidth", 1000):
    display(
                df[mask][['market', 'title', 'price', 'price_from', 'price_to', 'quantity', 'discountPercent']]
                .assign(fract=fraction)
                .sort_values(by='fract', ascending=True)
                .head(3)
            )

,market,title,price,price_from,price_to,quantity,discountPercent,fract
1788,Магнит,Драже M&Ms с соленым арахисом и молочным шоколадом 45г,2999,4499,<NA>,45.0,51,666
1867,Магнит,Драже M&MS Соленый арахис 145г,<NA>,<NA>,12999,145.0,30,896
1858,Магнит,Драже M&Ms арахис соленый 80г,7599,8099,<NA>,80.0,21,949


In [132]:

mask_su2 = df['su2'] == 'Майонез и соусы'
# mask_su3 = df['su3'] == 'Кетчуп'
mask1 = df['title'].str.lower().str.match('.*(помидорка).*').fillna(False)
# mask2 = df['quantity'] > 250
mask = (
    mask_su2
    # mask_su3
    & mask1
    # & mask2
    )
with pd.option_context("display.max_columns", 500, "display.max_colwidth", 1000):
    display(
                df[mask][['market', 'title', 'price', 'price_from', 'price_to', 'quantity', 'discountPercent']]
                .assign(fract=fraction)
                .sort_values(by='fract', ascending=True)
                .head(30)
            )


,market,title,price,price_from,price_to,quantity,discountPercent,fract
19558,Линия,Кетчуп Помидорка Острый 350г дой-пак,<NA>,<NA>,8999,350.0,26,257
19560,Линия,Кетчуп Помидорка Томатный 350г дой-пак,<NA>,<NA>,8999,350.0,26,257
19563,Линия,Кетчуп Помидорка Шашлычный 350г дой-пак,<NA>,<NA>,8999,350.0,26,257
847,Перекрёсток,"Соус ПОМИДОРКА Томатный с пряными травами; Кавказский; Краснодарский, 490 г",<NA>,<NA>,14999,490.0,22,306


In [131]:
mask_su2 = df['su2'] == 'Майонез и соусы'
# mask_su3 = df['su3'] == 'Кетчуп'
# mask1 = df['title'].str.lower().str.match('.*((рогачевъ).*(цельное)|(цельное).*(рогачевъ)).*').fillna(False)
mask1 = df['title'].str.lower().str.match('.*(рогачевъ).*').fillna(False)
# mask2 = df['quantity'] > 250
mask = (
    # mask_su2
    # mask_su3
    mask1
    # & mask2
    )
with pd.option_context("display.max_columns", 500, "display.max_colwidth", 1000):
    display(
                df[mask][['market', 'title', 'price', 'price_from', 'price_to', 'quantity', 'discountPercent']]
                .assign(fract=fraction)
                .sort_values(by='fract', ascending=True)
                .head(30)
            )

,market,title,price,price_from,price_to,quantity,discountPercent,fract
14995,Лента Гипермаркет,"Молоко сгущенное Рогачевъ цельное с сахаром 8.5%, 380 г, металлическая банка",<NA>,<NA>,11989,380.0,0,315
3698,Магнит,Молоко сгущенное Рогачевъ Вареное с сахаром 8.5% 360г,11599,13499,<NA>,360.0,36,322
13254,ВкусВилл,"Молоко сгущенное Рогачевъ цельное с сахаром 8,5% 380 г",<NA>,<NA>,13500,380.0,19,355
11229,Магнит Семейный,Молоко сгущенное Рогачевъ 8.5% 270г,<NA>,<NA>,10999,270.0,32,407
11997,Перекресток-Экспресс,"Молоко сгущённое Рогачевъ с сахаром 8.5%, 380г",<NA>,<NA>,15999,380.0,0,421
3644,Магнит,Молоко сгущенное Рогачевъ 8.5% 270г,<NA>,<NA>,11999,270.0,26,444
11981,Перекресток-Экспресс,"Молоко сгущённое Рогачевъ цельное с сахаром 8.5%, 270г",<NA>,<NA>,14499,270.0,0,537
27704,Ашан,"Молоко сгущенное «Рогачевъ» с сахаром ГОСТ 8,5% БЗМЖ, 270 г",<NA>,<NA>,17299,270.0,0,640
7210,Пятёрочка,Молоко сгущенное Рогачевъ цельное с сахаром 8.5%,15000,15500,<NA>,<NA>,0,<NA>
